# Cài đạt Các Thư viện cần thiết
# Sử dụng: PyTorch + OpenCV
# Dataset: FER2013

# download FER2013

In [ ]:
import kagglehub
path = kagglehub.dataset_download("msambare/fer2013")
print("Path to dataset files:", path)

In [1]:
!pip install opencv-python torch torchvision


  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached torchvision-0.24.1-cp312-cp312-win_amd64.whl.metadata (5.9 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached torch-2.9.1-cp312-cp312-win_amd64.whl.metadata (30 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached torchvision-0.24.1-cp312-cp312-win_amd64.whl (4.3 MB)
Using cached torch-2.9.1-cp312-cp312-win_amd64.whl (110.9 MB)

  Attempting uninstall: numpy

    Found existing installation: numpy 1.26.4

    Uninstalling numpy-1.26.4:

      Successfully uninstalled numpy-1.26.4

   ---------------------------------------- 0/4 [numpy]
   ---------------------------------------- 0/4 [numpy]
   ---------------------------------------- 0/4 [numpy]
   ---------------------------------------- 0/4 [numpy]
   ---------------------------------------- 0/4 [numpy]
   --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.26 requires langchain-core<1.0.0,>=0.3.66, but you have langchain-core 0.2.37 which is incompatible.
langchain-pinecone 0.2.8 requires langchain-core<1.0.0,>=0.3.34, but you have langchain-core 0.2.37 which is incompatible.
langchain-tests 0.3.22 requires langchain-core<2.0.0,>=0.3.77, but you have langchain-core 0.2.37 which is incompatible.
pandas 2.2.0 requires numpy<2,>=1.26.0; python_version >= "3.12", but you have numpy 2.2.6 which is incompatible.
scipy 1.12.0 requires numpy<1.29.0,>=1.22.4, but you have numpy 2.2.6 which is incompatible.


In [2]:
import cv2
img=cv2.imread(r"img_1.png")

# Thiết lập Model Và Huấn Luyện Mô Hình

## 1. Tiền Xử lý dữ liệu

In [ ]:
import os
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

In [ ]:
data_dir = r"D:\FINE-GRAINED-EMOTION-PREDICT\datasets\msambare\fer2013\versions\1"

transform = transforms.Compose([
    transforms.Grayscale(),            # 1 channel
    transforms.Resize((48,48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=False)

## 2. Setup GPU , Labels

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
num_epochs = 15
lr = 0.001
size_img = 48
model_path = "fer_cnn.pth"
Labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

## 3.Tạo Model FER_CNN

In [ ]:
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

class FER_CNN(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(128*12*12, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

## 4. Huấn Luyện Mô Hình

In [ ]:
model = FER_CNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0, 0, 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    correct_test, total_test = 0,0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct_test += (preds == labels).sum().item()
            total_test += labels.size(0)
    val_acc = correct_test / total_test
    print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, TrainAcc={train_acc:.4f}, ValAcc={val_acc:.4f}")

# Lưu model
torch.save(model.state_dict(), model_path)
print(f"Model đã lưu: {model_path}")

## 5.RealTime Webcam

In [ ]:
cap = cv2.VideoCapture(0)
model.eval()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x,y,w,h) in faces:
        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (size_img, size_img))
        roi = transform(roi).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(roi)
            _, pred = torch.max(output,1)
            emotion = Labels[pred.item()]
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    cv2.imshow("FER Webcam", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()